In [13]:
import csv
import numpy as np
from numpy import loadtxt
import pandas as pd

In [2]:
bus = 73
samples = 20000
Bus_data = loadtxt('BusData.dat')
Gen_data = loadtxt('GenData.dat')
Bra_data = loadtxt('BranchData.dat')
# Pred_data = loadtxt(('PF_Predict_%d.csv' %bus), delimiter=',')
Pg_data   = loadtxt(('Pg_Profile_%d.csv' %bus), delimiter=',')
PF_data   = loadtxt(('PF_Profile_%d.csv' %bus), delimiter=',')
PD_data   = loadtxt(('PD_Profile_%d.csv' %bus), delimiter=',')
TC_data   = loadtxt(('TC_Profile_%d.csv' %bus), delimiter=',')

In [3]:
PD_data.shape

(20000, 73)

In [4]:
index1 = int(0.8*samples)
index2 = int(0.1*samples)
index3 = index1 + index2

idxs = range(index3,samples)

In [5]:
from pyomo.environ import *

# instance the pyomo abstract model
model = AbstractModel()

### Set
model.BUS    = Set(initialize = Bus_data[:,0]-1)                        #k
model.BRANCH = Set(initialize = Bra_data[:,0]-1)                        #j
model.GEN    = Set(initialize = Gen_data[:,0]-1)                        #i

### Param
#Bus param = k
model.bus_num  = Param(model.BUS, initialize = Bus_data[:,1]-1)

load_profile = PD_data[0,:]
def bus_Pd(model, k):
    return load_profile[int(k)]
model.bus_Pd   = Param(model.BUS, initialize = bus_Pd)# mutable = True)

model.bus_type = Param(model.BUS, initialize = Bus_data[:,3])


#Gen param = i
model.gen_num  = Param(model.GEN, initialize = Gen_data[:,1]-1)

model.gen_bus  = Param(model.GEN, initialize = Gen_data[:,2]-1)

model.gen_Pmax = Param(model.GEN, initialize = Gen_data[:,3])#, mutable=True)

model.gen_Pmin = Param(model.GEN, initialize = Gen_data[:,4])#, mutable=True)

model.gen_C    = Param(model.GEN, initialize = Gen_data[:,5])



#Branch param = j
model.branch_num   = Param(model.BRANCH, initialize = Bra_data[:,1]-1)

model.branch_fbus  = Param(model.BRANCH, initialize = Bra_data[:,2]-1)

model.branch_tbus  = Param(model.BRANCH, initialize = Bra_data[:,3]-1)

model.branch_x     = Param(model.BRANCH, initialize = Bra_data[:,4])

model.branch_rateA = Param(model.BRANCH, initialize = Bra_data[:,5])#, mutable=True)

#base MVA
model.Base = Param(default=100.0)

### Var
model.bus_angle  = Var(model.BUS)
model.gen_supply = Var(model.GEN)
model.line_flow  = Var(model.BRANCH)

### Objective function
def objfunction(model,i):
    totalcost = sum(model.gen_supply[i]*model.gen_C[i] for i in model.GEN) #* model.BaseMVA
    return totalcost

model.cost = Objective(rule=objfunction, sense=minimize)


### constraint
#Power balance equation
def PowerBal(model,k):
    buspower = sum(model.line_flow[j] for j in model.BRANCH if model.branch_tbus[j] == k) - sum(model.line_flow[j] for j in model.BRANCH if model.branch_fbus[j] ==k) + sum(model.gen_supply[i] for i in model.GEN if model.gen_bus[i]==k) - model.bus_Pd[k]
    return buspower == 0
model.cons_PowerBal = Constraint(model.BUS,rule=PowerBal)

#power output limitation for generator
def PGenMax(model,i):
    return model.gen_supply[i] <= model.gen_Pmax[i]
model.cons_PGenMax = Constraint(model.GEN,rule=PGenMax)
def PGenMin(model,i):
    return model.gen_Pmin[i] <=  model.gen_supply[i]
model.cons_PGenMin = Constraint(model.GEN,rule=PGenMin)


#line flow equation
def lineflow(model,j):
    return model.line_flow[j] == model.Base * ((model.bus_angle[model.branch_fbus[j]]-model.bus_angle[model.branch_tbus[j]])/model.branch_x[j])
model.cons_lineflow = Constraint(model.BRANCH, rule=lineflow)

#thermal constraint
def ThermalMax(model,j):
    return model.line_flow[j] <= model.branch_rateA[j]
model.cons_ThermalMax = Constraint(model.BRANCH, rule=ThermalMax)
def ThermalMin(model,j):
    return -model.branch_rateA[j] <=  model.line_flow[j] 
model.cons_ThermalMin = Constraint(model.BRANCH, rule=ThermalMin)



In [6]:
# form the pyomo data file
# form_data.formdata()
# instance according on the dat file

### set the solver
solver = SolverFactory('solver\gurobi.exe')
dcopf = model.create_instance()

### Suppress infeasible error warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
# load_profile = PD_data[idxs,:]

Pd_List = np.zeros([index2,Bus_data.shape[0]])
Pg_List = np.zeros([index2,Gen_data.shape[0]])
PF_List = np.zeros([index2,Bra_data.shape[0]])
TC_List = np.zeros([index2,1])
Optimal = np.zeros([index2,1])



In [7]:
import time
run = 10
total_t = np.zeros(run)

for t in range(run):
    
    SampleT = np.zeros([index2])
    ### Run solver to generate data samples
    m = index3
    n = 0


    while m < samples:
        load_profile = PD_data[m,:]

        #Solving the model and record time
        dcopf = model.create_instance()
        start_t = time.time()
        results = solver.solve(dcopf, tee=False)
        stop_t = time.time()  
        SampleT[n] = stop_t - start_t
        
        if results.solver.termination_condition == "optimal":       
            Pg_List[n,:] = dcopf.gen_supply[:]()
            PF_List[n,:] = dcopf.line_flow[:]()
            TC_List[n,:] = dcopf.cost[:]()
            Optimal[n,:] = 1
        m = m+1
        n = n+1

        if m % 100 == 0:
            print(m)



    total_t[t] = np.sum(SampleT)
    #Total amount of time
    print('Total amount of time is', total_t[t])

18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 130.89997839927673
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 135.87120985984802
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 127.18240237236023
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 123.45293617248535
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 123.56496119499207
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
Total amount of time is 123.28788113594055
18100
18200
18300
1840

In [8]:
print('Average amount of time is', np.mean(total_t))
print(total_t)

Average amount of time is 125.86610991954804
[130.8999784  135.87120986 127.18240237 123.45293617 123.56496119
 123.28788114 122.74729824 125.57004023 122.9284873  123.15590429]


In [ ]:
pd_Time = pd.DataFrame(run_t)
pd_Time.to_excel('Result/Time_GNN.xlsx')

In [9]:
#Validate data for Total Cost, and Power Flow
print(np.sum(PF_List == PF_data[idxs,:]))
print(np.sum(TC_List == TC_data[idxs]))

216000
2000


In [14]:
pd_Time = pd.DataFrame(total_t)
pd_Time.to_excel('Result/Time_Baseline.xlsx')